In [5]:
from lively import  Solver, MoveShape,AddShape, Translation, Rotation,Transform,SmoothnessMacroObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Define box Shape, addShape, and Collision

box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=1.0, y =2.0, z=3.0),rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
addShape = AddShape(id = "123", shape = box )
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objective (smoothness macro)
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {},weights = {},time = 0.0,shape_updates = [addShape])
# Log the initial statse
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

({'y': 0.0, 'x': 0.0, 'z': 0.0}, {'w': 1.0, 'y': 0.0, 'x': 0.0, 'z': 0.0})
{'panda_joint7': -0.00014566663990785232, 'panda_joint6': 1.8675023747935595, 'panda_joint4': -1.570839317425915, 'panda_finger_joint1': 0.02921128999354055, 'panda_finger_joint2': 0.02921128999354055, 'panda_joint1': -1.7631091301343767e-07, 'panda_joint2': -4.544676171770894e-06, 'panda_joint5': -3.900486124027599e-08, 'panda_joint3': -1.7526311293639713e-07}
TransformInfo: world:{translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}, local: {translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}
[ProximityInfo: {shape1: "panda_link1", shape2: "panda_link2", distance: -0.04999999999998844, points: Some(([-2.675138042951722e-7, 1.2504080936253242e-13, 0.39299999999939755], [-5.061352772766245e-8, 9.069134332406747e-14, 0.3429999999998796])), physical: true, loss: 1.0499999999999885, average_distance: Some(1.0)}, ProximityInfo: {shape1: "panda_link3", shape2: "panda_lin